<a href="https://colab.research.google.com/github/ghr8635/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/blob/main/point_pillar_based_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Install Required Libraries**

In [ ]:
pip install mmdet3d
pip install torch torchvision mmcv open3d



**Import Libraries**

In [ ]:
import torch
import numpy as np
import open3d as o3d
from mmdet3d.datasets.pipelines import Compose
from mmdet3d.models import build_model
from mmcv import Config


**Preprocess the .pcd Data**

In [ ]:
# Function to convert .pcd to .bin
def convert_pcd_to_bin(pcd_file, bin_file):
    pcd = o3d.io.read_point_cloud(pcd_file)  # Read the PCD file
    points = np.asarray(pcd.points)  # Get the points (XYZ coordinates)
    points.tofile(bin_file)  # Save the points to a binary file

# Example usage
convert_pcd_to_bin("path_to_your_data.pcd", "path_to_your_output_data.bin")

**Prepare the Dataset Pipeline**

In [ ]:
# Load the configuration file for PointPillars
cfg = Config.fromfile('configs/pointpillars/pointpillars_kitti.py')

# Modify the configuration to remove detection head (we are only extracting features)
cfg.model.bbox_head = None

# Set the paths to your dataset (adjust the paths to your specific dataset)
cfg.data.train.data_root = 'path/to/your_dataset/training/'
cfg.data.test.data_root = 'path/to/your_dataset/testing/'

# Define the preprocessing pipeline (for test phase)
test_pipeline = Compose(cfg.data.test.pipeline)


**Build the PointPillars Model**

In [ ]:
# Build the model using the configuration
model = build_model(cfg.model)

# Load a pre-trained checkpoint (optional, you can skip this if training from scratch)
checkpoint = 'path_to_pretrained_checkpoint.pth'  # If available, you can use a pre-trained checkpoint
model.load_checkpoint(checkpoint)

# Set the model to evaluation mode (since we are only extracting features)
model.eval()


**Process the data from .bin format to feed to the model**

In [ ]:
# Load the point cloud data (after converting to .bin format)
pcd_data = np.fromfile("path_to_your_output_data.bin", dtype=np.float32).reshape(-1, 4)  # (N, 4) format

# Create a dictionary with the points data
data = dict(points=pcd_data)

# Apply the preprocessing pipeline
processed_data = test_pipeline(data)

# Extract the processed point cloud data
processed_points = processed_data['points'].unsqueeze(0)  # Add batch dimension


**Extract Features Using PointPillars**

In [ ]:
# Forward pass through the model to extract features (no bbox head, just the backbone)
with torch.no_grad():
    extracted_features = model.extract_feat(processed_points)

# Output the shape of the extracted features (this will be a tensor)
print(f"Extracted feature shape: {extracted_features.shape}")


**Save the model**

In [ ]:
# Save the entire model (architecture + weights)
torch.save(model, 'pointpillars_feature_extractor.pth')

